In [ ]:
# Install required packages (only if needed)
!pip install -q tensorflow

# Import necessary libraries
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

In [3]:
# Define dataset directory paths
dataset_path = "/kaggle/input/fer2013"
train_dir = os.path.join(dataset_path, "train")
test_dir = os.path.join(dataset_path, "test")

In [4]:
# Define Data Augmentation with mild transformations
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,  
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators without 'workers' (Handled by Keras automatically)
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(48, 48), 
    color_mode="grayscale",
    batch_size=64, 
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(48, 48), 
    color_mode="grayscale",
    batch_size=64, 
    class_mode="categorical"
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [5]:
# Compute class weights to handle imbalanced dataset
train_classes = np.array(train_generator.classes)
class_labels = np.unique(train_classes)
class_weights = compute_class_weight('balanced', classes=class_labels, y=train_classes)
class_weights = np.clip(class_weights, 0.5, 2.0)  # Lower max cap to 2.0 to prevent extreme bias
class_weights_dict = dict(enumerate(class_weights))

print("Final Class Weights:", class_weights_dict)

Final Class Weights: {0: 1.0266046844269623, 1: 2.0, 2: 1.0010460615781582, 3: 0.5684387684387684, 4: 0.8260394187886635, 5: 0.8491274770777877, 6: 1.293372978330405}


In [6]:
from tensorflow.keras.regularizers import l2

# Define an improved CNN architecture
model = Sequential([
    Input(shape=(48,48,1)),

    Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(1e-4)),
    Dropout(0.4),
    Dense(128, activation='relu', kernel_regularizer=l2(1e-4)),
    Dropout(0.3),
    Dense(7, activation='softmax')  # 7 Emotion Classes
])

In [7]:
# Define optimizer with weight decay
optimizer = AdamW(learning_rate=3e-4, weight_decay=1e-4)

# Compile model with label smoothing
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),  
    optimizer=optimizer,
    metrics=["accuracy"]
)

# Show model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 48, 48, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 48, 48, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 48, 48, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 24, 24, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 24, 24, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 24, 24, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 24, 24, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 12, 12, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 12, 12, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 12, 12, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 9216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       4,719,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,931,719 (22.63 MB)

 Trainable params: 5,930,823 (22.62 MB)

 Non-trainable params: 896 (3.50 KB)

In [8]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=30,
    class_weight=class_weights_dict,
    callbacks=[early_stopping, lr_scheduler]
)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


449/449 ━━━━━━━━━━━━━━━━━━━━ 257s 542ms/step - accuracy: 0.1875 - loss: 2.0405 - val_accuracy: 0.2386 - val_loss: 2.0232 - learning_rate: 3.0000e-04
Epoch 2/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 47s 103ms/step - accuracy: 0.2242 - loss: 1.8059 - val_accuracy: 0.2619 - val_loss: 1.9049 - learning_rate: 3.0000e-04
Epoch 3/30
117/449 ━━━━━━━━━━━━━━━━━━━━ 30s 92ms/step - accuracy: 0.2473 - loss: 1.7711

In [ ]:
def plot_metrics(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    ax1.plot(history.history['accuracy'], label='Train Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Val Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')
    ax1.legend()

    ax2.plot(history.history['loss'], label='Train Loss')
    ax2.plot(history.history['val_loss'], label='Val Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')
    ax2.legend()

    plt.show()

plot_metrics(history)

In [ ]:
import random

# Pick a random test image
random_index = random.randint(0, len(test_generator.filenames)-1)
img_path = os.path.join(test_dir, test_generator.filenames[random_index])

# Load and preprocess the image
def predict_emotion(img_path, model):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (48,48))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=-1)

    prediction = model.predict(img)
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
    predicted_emotion = emotion_labels[np.argmax(prediction)]
    
    plt.imshow(cv2.imread(img_path))
    plt.title(f"Predicted Emotion: {predicted_emotion}")
    plt.axis("off")
    plt.show()

predict_emotion(img_path, model)

In [ ]:
#MOOD MAPPING
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Emotion to Mood Mapping (Expanded for 7 emotions)
# These weights are examples and can be fine-tuned
emotion_to_mood_mapping = {
    'happy': {'happy': 1.0, 'energetic': 0.7, 'relaxed': 0.3},
    'sad': {'sad': 1.0, 'relaxed': 0.5},
    'angry': {'energetic': 0.8, 'sad': 0.2},
    'neutral': {'relaxed': 0.6, 'happy': 0.2, 'sad': 0.2},
    'surprise': {'energetic': 0.9, 'happy': 0.5},
    'fear': {'sad': 0.7, 'relaxed': 0.3},
    'disgust': {'sad': 0.8, 'relaxed': 0.2}
}

In [ ]:
def map_emotions_to_mood_scores(emotion_probabilities, emotion_labels, emotion_to_mood_map):
    """
    Maps emotion probabilities to mood scores.

    Args:
        emotion_probabilities (np.array): Probabilities for each emotion class.
        emotion_labels (list): List of emotion labels corresponding to the probabilities.
        emotion_to_mood_map (dict): Mapping from emotion labels to mood scores.

    Returns:
        dict: Normalized mood scores.
    """
    mood_scores = {
        "happy": 0.0,
        "sad": 0.0,
        "energetic": 0.0,
        "relaxed": 0.0,
        "romantic": 0.0
    }

    for i, prob in enumerate(emotion_probabilities):
        emotion_label = emotion_labels[i]
        if emotion_label in emotion_to_mood_map:
            for mood, weight in emotion_to_mood_map[emotion_label].items():
                mood_scores[mood] += prob * weight

    # Normalize mood scores to sum to 100
    total_score = sum(mood_scores.values())
    if total_score > 0:
        mood_scores = {mood: (score / total_score) * 100 for mood, score in mood_scores.items()}
    return mood_scores

In [ ]:
def final_mood(face_mood, bg_mood, text_mood):
    """
    Combines mood scores from face, background, and text.
    """
    combined = {}
    for mood in face_mood:
        combined[mood] = 0.5 * face_mood[mood] + 0.3 * text_mood[mood] + 0.2 * bg_mood[mood]
    total = sum(combined.values())
    return {mood: (score / total) * 100 for mood, score in combined.items()}


def mapping_values(mood_scores_test, mood_feature_mapping_test):    
    """
    Maps mood scores to a final mood vector.
    """
    scaler = MinMaxScaler()
    mood_feature_matrix = np.array(list(mood_feature_mapping_test.values()))
    mood_feature_matrix = scaler.fit_transform(mood_feature_matrix)

    for i, mood in enumerate(mood_feature_mapping_test.keys()):
        mood_feature_mapping_test[mood] = mood_feature_matrix[i].tolist()

    num_features = 8 
    final_mood_vector = np.zeros(num_features)

    for mood, weight in mood_scores_test.items():
        mood_vector = np.array(mood_feature_mapping_test[mood])
        final_mood_vector += weight * mood_vector 

    final_mood_vector /= sum(mood_scores_test.values()) 
    return final_mood_vector


mood_feature_mapping = {
    "happy":         [0.8, 0.7, 0.8, -5, 0.1, 0.1, 0.0, 140],
    "sad":           [0.2, 0.2, 0.3, -20, 0.05, 0.8, 0.3, 80],
    "energetic":     [0.6, 0.9, 0.9, -3, 0.2, 0.1, 0.0, 160],
    "relaxed":       [0.5, 0.4, 0.4, -15, 0.05, 0.7, 0.5, 90],
    "romantic":      [0.7, 0.5, 0.6, -8, 0.1, 0.5, 0.4, 100],
}

In [ ]:
# --- Prediction and Recommendation Flow ---

# Load the trained model
try:
    model = load_model("fer2013_cnn_improved.h5")
except Exception as e:
    print(f"Error loading model: {e}. Please ensure 'fer2013_cnn_improved.h5' exists after training.")
    exit()

# Get emotion labels from the train generator
emotion_labels = list(train_generator.class_indices.keys())

# Example: Predict emotion for a dummy image (replace with actual image input)
# In a real application, you would load an image, preprocess it, and then predict
dummy_image_path = os.path.join(test_dir, 'happy', os.listdir(os.path.join(test_dir, 'happy'))[0]) # Example: pick a happy image
img = cv2.imread(dummy_image_path, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (48, 48))
img = np.expand_dims(img, axis=-1)
img = np.expand_dims(img, axis=0)
img = img / 255.0

emotion_probabilities = model.predict(img)[0]
print(f"Emotion Probabilities: {emotion_probabilities}")

# Map predicted emotions to mood scores
face_mood_scores = map_emotions_to_mood_scores(emotion_probabilities, emotion_labels, emotion_to_mood_mapping)
print(f"Face Mood Scores: {face_mood_scores}")

# For simplicity, let's assume dummy values for bg_mood and text_mood.
# In a full system, these would come from other analysis modules.
bg_mood_dummy = {
    "happy": 0.1,
    "sad": 0.1,
    "energetic": 0.3,
    "relaxed": 0.4,
    "romantic": 0.1
}
text_mood_dummy = {
    "happy": 0.2,
    "sad": 0.1,
    "energetic": 0.4,
    "relaxed": 0.2,
    "romantic": 0.1
}

# Combine moods
combined_mood_scores = final_mood(face_mood_scores, bg_mood_dummy, text_mood_dummy)
print(f"Combined Mood Scores: {combined_mood_scores}")

# Generate the final mood vector
final_mood_vector = mapping_values(mood_feature_mapping_test=mood_feature_mapping, mood_scores_test=combined_mood_scores)
print(f"Final Mood Vector: {final_mood_vector}")

# Load the song dataset
df = pd.read_csv("/kaggle/input/moodify/updated_moodify_dataset.csv")

features = ["valence", "energy", "danceability", "loudness", "speechiness",
            "acousticness", "instrumentalness", "tempo"]

song_vectors = df[features].values

# Calculate cosine similarities
similarities = cosine_similarity([final_mood_vector], song_vectors)

# Get top 5 recommended songs
top_indices = np.argsort(similarities[0])[::-1][:5]
recommended_songs = df.iloc[top_indices]
print("\nRecommended Songs:")
print(recommended_songs[["uri","song_name"]])